In [ ]:
## USER 테이블 build
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
spark = (
    SparkSession.builder \
                .master("local")
                .appName("World Count")
                .getOrCreate()
)
file_path = r"C:\Users\brian\Desktop\JUNSOO\Project\ETL_DATA\data\ranking_2024-10-11.json"
df = spark.read.format('json').option("multiLine", True).load(file_path)

# 직업명을 제대로 채우기
import pyspark.sql.functions as F
df = df.select(F.explode("ranking").alias("USER"))
df = df.select("USER.character_name",
               "USER.date",
               "USER.class_name",
               "USER.sub_class_name",
               "USER.character_level",
               "USER.character_exp",
               "USER.ranking")
df = df.withColumn("class",df["sub_class_name"])
df = df.withColumn("class",F.when(df["sub_class_name"]== "", df["class_name"]) \
                            .otherwise(df["class"]))
df = df.drop("class_name","sub_class_name")


# 각 유저의 위치한 지역 컬럼추가
df = df.withColumn("status",
                   F.when(df["character_level"]>=290,"Tallahart") \
                    .when((df["character_level"]<=289)&(df["character_level"]>=285),"Carcion") \
                    .when((df["character_level"]<=284)&(df["character_level"]>=280),"Arteria") \
                    .when((df["character_level"]<=279)&(df["character_level"]>=275),"Dowonkyung") \
                    .when((df["character_level"]<=274)&(df["character_level"]>=270),"Odium") \
                    .when((df["character_level"]<=269)&(df["character_level"]>=265),"HotelArcs") \
                    .when((df["character_level"]<=264)&(df["character_level"]>=260),"Cernium") \
                    .otherwise("AcaneRiver"))


[Row(character_name='상남자전사', date='2024-10-11', character_level=276, character_exp=7679468622837, ranking=29991, class='히어로', status='Dowonkyung'),
 Row(character_name='추나선생님', date='2024-10-11', character_level=276, character_exp=7674026029003, ranking=29992, class='바이퍼', status='Dowonkyung'),
 Row(character_name='카인열렸따아', date='2024-10-11', character_level=276, character_exp=7671634905232, ranking=29993, class='카인', status='Dowonkyung'),
 Row(character_name='칠흑먹자나비', date='2024-10-11', character_level=276, character_exp=7669184826707, ranking=29994, class='비숍', status='Dowonkyung'),
 Row(character_name='세로쉬', date='2024-10-11', character_level=276, character_exp=7666392451590, ranking=29995, class='아델', status='Dowonkyung'),
 Row(character_name='무당기린', date='2024-10-11', character_level=276, character_exp=7665301903843, ranking=29996, class='라라', status='Dowonkyung'),
 Row(character_name='평상복', date='2024-10-11', character_level=276, character_exp=7657965070591, ranking=29997, class=